### 1. Import and install dependencies

In [23]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [2]:
#Houlistic Models
mp_holistic = mp.solutions.holistic 

#Drawing Utilities
mp_drawing = mp.solutions.drawing_utils


### Media Pipe Detection from the rendered images

In [3]:
def media_pipe_detection(image,model):
    # color convertion
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    
    # setting the image to unwritable 
    image.flags.writeable = False
    
    # make prediction using the model
    results = model.process(image)
    image.flags.writeable = True

    # color convertion
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

### Update the image realtime with the data point optioned from media pipe

In [5]:

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

### Module to test the web cam and media pipe

In [6]:
#access the web cam
def testing_web_cam():
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            #feed from the web cam
            ret,frame = cap.read()
            image, results = media_pipe_detection(frame, holistic)

            draw_styled_landmarks(image,results)

            #display the frame captured
            cv2.imshow('OpenCV feed',image)
    # 
            #break the web capture if the 'q' key pressed on keyboard
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        # release the web cam
        cap.release()
        cv2.destroyAllWindows()

In [7]:

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


### Create the folder structure for the storing the data

In [8]:
DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['hello', 'thanks', 'iloveyou'])

no_sequences = 30

sequence_length = 30

In [9]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


### Collecting the test data for 3 actions to train the model

In [10]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = media_pipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [11]:
label_map = {label:num for num, label in enumerate(actions)}


### Pre processing training data 

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequences), to_categorical(labels).astype(int), test_size=0.05)


In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

### Training model with LSTM 

In [15]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
model.summary()


Epoch 1/2000
3/3 [==============================] - 5s 393ms/step - loss: 1.7728 - categorical_accuracy: 0.2524
Epoch 2/2000
3/3 [==============================] - 1s 156ms/step - loss: 6.2301 - categorical_accuracy: 0.3307
Epoch 3/2000
3/3 [==============================] - 0s 116ms/step - loss: 3.0297 - categorical_accuracy: 0.2388
Epoch 4/2000
3/3 [==============================] - 0s 112ms/step - loss: 3.4695 - categorical_accuracy: 0.3346
Epoch 5/2000
3/3 [==============================] - 0s 107ms/step - loss: 6.9291 - categorical_accuracy: 0.3013
Epoch 6/2000
3/3 [==============================] - 0s 120ms/step - loss: 6.7977 - categorical_accuracy: 0.2407
Epoch 7/2000
3/3 [==============================] - 0s 98ms/step - loss: 2.6330 - categorical_accuracy: 0.4128
Epoch 8/2000
3/3 [==============================] - 0s 108ms/step - loss: 2.6155 - categorical_accuracy: 0.3424
Epoch 9/2000
3/3 [==============================] - 0s 113ms/step - loss: 1.5882 - categorical_accuracy: 

3/3 [==============================] - 0s 133ms/step - loss: 0.6074 - categorical_accuracy: 0.7240
Epoch 74/2000
3/3 [==============================] - 0s 127ms/step - loss: 0.8493 - categorical_accuracy: 0.5793
Epoch 75/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.6535 - categorical_accuracy: 0.6301
Epoch 76/2000
3/3 [==============================] - 0s 109ms/step - loss: 0.7789 - categorical_accuracy: 0.5284
Epoch 77/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.8433 - categorical_accuracy: 0.7376
Epoch 78/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.8939 - categorical_accuracy: 0.6928
Epoch 79/2000
3/3 [==============================] - 0s 104ms/step - loss: 0.8259 - categorical_accuracy: 0.5576
Epoch 80/2000
3/3 [==============================] - 0s 104ms/step - loss: 0.7149 - categorical_accuracy: 0.6751
Epoch 81/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.7092 - categorical_accuracy: 0.63

3/3 [==============================] - 0s 113ms/step - loss: 0.2625 - categorical_accuracy: 0.9295
Epoch 146/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.2743 - categorical_accuracy: 0.9002
Epoch 147/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.2097 - categorical_accuracy: 0.9295
Epoch 148/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.1736 - categorical_accuracy: 0.9432
Epoch 149/2000
3/3 [==============================] - 0s 114ms/step - loss: 0.2733 - categorical_accuracy: 0.8963
Epoch 150/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.2209 - categorical_accuracy: 0.9139
Epoch 151/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.2178 - categorical_accuracy: 0.9139
Epoch 152/2000
3/3 [==============================] - 0s 107ms/step - loss: 0.2399 - categorical_accuracy: 0.8924
Epoch 153/2000
3/3 [==============================] - 0s 112ms/step - loss: 0.1829 - categorical_accuracy

3/3 [==============================] - 0s 100ms/step - loss: 0.0887 - categorical_accuracy: 0.9706
Epoch 218/2000
3/3 [==============================] - 0s 101ms/step - loss: 0.1096 - categorical_accuracy: 0.9491
Epoch 219/2000
3/3 [==============================] - 0s 112ms/step - loss: 0.0648 - categorical_accuracy: 0.9902
Epoch 220/2000
3/3 [==============================] - 0s 117ms/step - loss: 0.0565 - categorical_accuracy: 0.9804
Epoch 221/2000
3/3 [==============================] - 0s 104ms/step - loss: 0.0655 - categorical_accuracy: 0.9824
Epoch 222/2000
3/3 [==============================] - 0s 110ms/step - loss: 0.0562 - categorical_accuracy: 0.9804
Epoch 223/2000
3/3 [==============================] - 0s 113ms/step - loss: 0.0705 - categorical_accuracy: 0.9550
Epoch 224/2000
3/3 [==============================] - 0s 109ms/step - loss: 0.0548 - categorical_accuracy: 0.9726
Epoch 225/2000
3/3 [==============================] - 0s 101ms/step - loss: 0.0670 - categorical_accura

3/3 [==============================] - 0s 98ms/step - loss: 0.1120 - categorical_accuracy: 0.9902
Epoch 290/2000
3/3 [==============================] - 0s 108ms/step - loss: 0.1395 - categorical_accuracy: 0.9491
Epoch 291/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.1249 - categorical_accuracy: 0.9726
Epoch 292/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.1562 - categorical_accuracy: 0.9608
Epoch 293/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.1129 - categorical_accuracy: 0.9589
Epoch 294/2000
3/3 [==============================] - 0s 106ms/step - loss: 0.1099 - categorical_accuracy: 0.9745
Epoch 295/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.0983 - categorical_accuracy: 0.9648
Epoch 296/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.0934 - categorical_accuracy: 0.9804
Epoch 297/2000
3/3 [==============================] - 0s 107ms/step - loss: 0.1306 - categorical_accuracy: 

...

3/3 [==============================] - 0s 108ms/step - loss: 1.0990 - categorical_accuracy: 0.3268
Epoch 1647/2000
3/3 [==============================] - 0s 112ms/step - loss: 1.0989 - categorical_accuracy: 0.3268
Epoch 1648/2000
3/3 [==============================] - 0s 104ms/step - loss: 1.0992 - categorical_accuracy: 0.3190
Epoch 1649/2000
3/3 [==============================] - 0s 109ms/step - loss: 1.0979 - categorical_accuracy: 0.3581
Epoch 1650/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.0988 - categorical_accuracy: 0.3307
Epoch 1651/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0967 - categorical_accuracy: 0.3893
Epoch 1652/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.0987 - categorical_accuracy: 0.3307
Epoch 1653/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.0994 - categorical_accuracy: 0.3112
Epoch 1654/2000
3/3 [==============================] - 0s 107ms/step - loss: 1.0979 - categorical

3/3 [==============================] - 0s 113ms/step - loss: 1.0977 - categorical_accuracy: 0.3659
Epoch 1718/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.0995 - categorical_accuracy: 0.3151
Epoch 1719/2000
3/3 [==============================] - 0s 104ms/step - loss: 1.1001 - categorical_accuracy: 0.2956
Epoch 1720/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.0989 - categorical_accuracy: 0.3307
Epoch 1721/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.0981 - categorical_accuracy: 0.3503
Epoch 1722/2000
3/3 [==============================] - 0s 114ms/step - loss: 1.0974 - categorical_accuracy: 0.3698
Epoch 1723/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.0992 - categorical_accuracy: 0.3190
Epoch 1724/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.0986 - categorical_accuracy: 0.3386
Epoch 1725/2000
3/3 [==============================] - 0s 106ms/step - loss: 1.0983 - categorica

3/3 [==============================] - 0s 105ms/step - loss: 1.0986 - categorical_accuracy: 0.3386
Epoch 1789/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.0997 - categorical_accuracy: 0.3034
Epoch 1790/2000
3/3 [==============================] - 0s 103ms/step - loss: 1.0980 - categorical_accuracy: 0.3581
Epoch 1791/2000
3/3 [==============================] - 0s 103ms/step - loss: 1.0973 - categorical_accuracy: 0.3776
Epoch 1792/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.0988 - categorical_accuracy: 0.3307
Epoch 1793/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.0987 - categorical_accuracy: 0.3347
Epoch 1794/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.0994 - categorical_accuracy: 0.3151
Epoch 1795/2000
3/3 [==============================] - 0s 97ms/step - loss: 1.0992 - categorical_accuracy: 0.3190
Epoch 1796/2000
3/3 [==============================] - 0s 106ms/step - loss: 1.0982 - categorical_

3/3 [==============================] - 0s 111ms/step - loss: 1.0991 - categorical_accuracy: 0.3229
Epoch 1860/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.0986 - categorical_accuracy: 0.3386
Epoch 1861/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.0973 - categorical_accuracy: 0.3776
Epoch 1862/2000
3/3 [==============================] - 0s 111ms/step - loss: 1.0980 - categorical_accuracy: 0.3542
Epoch 1863/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.0986 - categorical_accuracy: 0.3386
Epoch 1864/2000
3/3 [==============================] - 0s 104ms/step - loss: 1.0988 - categorical_accuracy: 0.3307
Epoch 1865/2000
3/3 [==============================] - 0s 105ms/step - loss: 1.0972 - categorical_accuracy: 0.3815
Epoch 1866/2000
3/3 [==============================] - 0s 112ms/step - loss: 1.0993 - categorical_accuracy: 0.3190
Epoch 1867/2000
3/3 [==============================] - 0s 100ms/step - loss: 1.0993 - categorical

3/3 [==============================] - 0s 121ms/step - loss: 1.1001 - categorical_accuracy: 0.2956
Epoch 1931/2000
3/3 [==============================] - 0s 107ms/step - loss: 1.0992 - categorical_accuracy: 0.3190
Epoch 1932/2000
3/3 [==============================] - 0s 102ms/step - loss: 1.0991 - categorical_accuracy: 0.3229
Epoch 1933/2000
3/3 [==============================] - 0s 101ms/step - loss: 1.0982 - categorical_accuracy: 0.3503
Epoch 1934/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0985 - categorical_accuracy: 0.3386
Epoch 1935/2000
3/3 [==============================] - 0s 113ms/step - loss: 1.0983 - categorical_accuracy: 0.3464
Epoch 1936/2000
3/3 [==============================] - 0s 99ms/step - loss: 1.0982 - categorical_accuracy: 0.3464
Epoch 1937/2000
3/3 [==============================] - 0s 103ms/step - loss: 1.0983 - categorical_accuracy: 0.3464
Epoch 1938/2000
3/3 [==============================] - 0s 110ms/step - loss: 1.0975 - categorical

### Saving the model for future reference

In [17]:
model.save('action.h5')

In [26]:
model.load_weights('action.h5')

### Confusion matrix and Accuracy score of the model

In [29]:
yhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)


array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]])

In [31]:
accuracy_score(ytrue, yhat)


1.0

### Testing the model with the realtime sign detection 

In [27]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [32]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = media_pipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hello
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.

<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'mediapipe.python.solution_base.SolutionOutputs'>
iloveyou
<class 'me